In [ ]:
from Tokenizer import Tokenizer
from data_module import DataModule
from Model import Transformer_LM
import torch

class Configs:
    def __init__(self):
        self.manifest_file = "total_am.txt"
        self.labels_path = "aihub_labels.csv"
        self.train_ratio = 0.8
        self.num_workers = 4
        self.batch_size = 64
        self.sample_mode = 'random' #'smart'
        self.teacher_forcing_ratio = 0.0
        
        self.num_classes = 2001
        self.d_model = 512
        self.d_ff = 2048
        self.num_heads = 4
        self.num_layers = 3
        self.model_name = "BERT"
        
configs = Configs()
tokenizer = Tokenizer(label_file=configs.labels_path)
tokenizer.idx2char[2000] = "<mask>"
model = Transformer_LM(
    num_classes=configs.num_classes,
    d_model=configs.d_model,
    d_ff=configs.d_ff,
    num_heads=configs.num_heads,
    num_layers=configs.num_layers,
    model=configs.model_name
)

In [ ]:
# Test GPT Model
model.load_state_dict(torch.load("gpt.pt"))

def text_generator(sentence: str, length: int):
    # sentence : 입력 문장
    # length : 몇 글자를 더 이어 쓸것인지
    inputs = tokenizer.encode(sentence)
    inputs = torch.Tensor(inputs).unsqueeze(0).long().cuda()
    
    with torch.no_grad():
        for i in range(length):
            # 문장 뒤에 아무 글자나 붙여서 다음 글자까지 예측하도록 함.
            blank = torch.Tensor([[5]]).cuda()
            inputs = torch.cat((inputs, blank), dim=1).long()
            input_lengths = torch.Tensor(inputs.size(1)).long().cuda()
            logits, preds = model.forward(inputs, input_lengths)
            
            # 앞의 글자들은 바꿀 것이 아니기 때문에 예측한 맨 마지막 글자만 뒤에 추가한다!
            inputs[0][-1] = preds[0][-1]
            print(tokenizer.decode(inputs))
    return tokenizer.decode(inputs)

In [ ]:
text_generator("이건 근의 공식을", 10)

In [ ]:
# Test BERT Model

model.load_state_dict(torch.load("bert.pt"))

def text_corrector(sentence: str, mask_num: int):
    # sentence : 입력 문장
    # mask_num : masking 할 문자 수
    mask_indexes = random.sample(range(len(sentence)), mask_num)

    #inputs = [tokenizer.sos_token] + tokenizer.encode(sample) + [tokenizer.eos_token]
    inputs = tokenizer.encode(sample)
    for idx in mask_indexes:
        inputs[idx] = 2000

    input_length = [len(inputs)]
    
    inputs = torch.Tensor(inputs).unsqueeze(0).int()
    inputs = inputs.cuda()

    logits, preds = model(inputs, input_length)

    for idx in mask_indexes:
        # <sos> 빼고
        inputs[0][idx] = preds[0][idx]

    return tokenizer.decode(inputs[0][1:-1])

In [ ]:
text_corrector("이렇게 좋은 날씨에 사무실에서 뭐하는 거람")